In [11]:
#pip install lxml

In [12]:
from bs4 import BeautifulSoup
import os

In [13]:
# Define output
output_def = {
    'alias':{
        'time':'pt_time',
        'lat':'position_lat',
        'long':'position_long',
        'alt':'alt_meters',
        'distance':'dist_meters',
        'hr':'hrm_bpm'
    },
    'header':['time','lat','long','alt','distance','hr']
}

In [14]:
def read_track(track_data):
    trk_pts = []
    # get all Trackpoint elements
    trackpoints = track_data.find_all('Trackpoint')
    for trackpoint in trackpoints:
        pt_data = {
            'pt_time':None,
            'position_lat':None,
            'position_long':None,
            'alt_meters':None,
            'dist_meters':None,
            'hrm_bpm':None
        }
        
        if trackpoint.Time is not None:
            pt_data['pt_time'] = trackpoint.Time.text
        if trackpoint.Position is not None:
            if trackpoint.Position.LatitudeDegrees is not None:
                pt_data['position_lat'] = trackpoint.Position.LatitudeDegrees.text
            if trackpoint.Position.LongitudeDegrees is not None:
                pt_data['position_long'] = trackpoint.Position.LongitudeDegrees.text
        if trackpoint.AltitudeMeters is not None:
            pt_data['alt_meters'] = trackpoint.AltitudeMeters.text
        if trackpoint.DistanceMeters is not None:
            pt_data['dist_meters'] = trackpoint.DistanceMeters.text
        if trackpoint.HeartRateBpm.Value is not None:
            pt_data['hrm_bpm'] = trackpoint.HeartRateBpm.Value.text
        trk_pts.append(pt_data)
    return trk_pts

In [15]:
# Read the file
# activity_17719094305_yoga.tcx (cardio/yoga)
# activity_17702547856.tcx (biking)
tcx_dir = '/home/jovyan/ceverhart-fitness_app/Fitness/'
for tcx_f in os.listdir(tcx_dir):
    tcx_path = os.path.join(tcx_dir,tcx_f)
    if os.path.isfile(tcx_path) and tcx_path[-3:].upper() == 'TCX':
        with open(tcx_path,'r') as tcx_file: 
            tcx_data = tcx_file.read()
            
            tcx_soup = BeautifulSoup(tcx_data, features="xml")

            sport = None
            activities = tcx_soup.find_all("Activity")
            for activity in activities:
                # Get the "Sport" attribute value
                if activity.has_attr('Sport'):
                    sport = activity['Sport']
                    print(sport)
                    laps = activity.find_all("Lap")
                    track_data = []
                    for lap in laps:
                        lap_data = lap.findChildren(recursive=False)
                        for child in lap_data:
                            # Get track data
                            if child.name == "Track":
                                track_data.extend(read_track(child))
                    # write the header to the screen
                    output_hdr = ''
                    for fld in output_def['header']:
                        output_hdr += fld + ','
                    print(output_hdr[:-1])
                    # Write track data to the screen
                    for pt in track_data:
                        output = ''
                        for fld in output_def['header']:
                            if pt[output_def['alias'][fld]] is None:
                                output += ','
                            else:
                                output += (pt[output_def['alias'][fld]]) + ','
                        print(output[:-1])

Biking
time,lat,long,alt,distance,hr
2024-12-06T15:07:08.000Z,38.888178979977965,-104.77269027382135,1936.800048828125,0.0,101
2024-12-06T15:07:09.000Z,38.888178979977965,-104.77269027382135,1936.800048828125,0.0,101
2024-12-06T15:07:10.000Z,38.888178979977965,-104.77269027382135,1936.800048828125,0.0,102
2024-12-06T15:07:11.000Z,38.88818199746311,-104.77269027382135,1936.800048828125,0.30000001192092896,102
2024-12-06T15:07:12.000Z,38.888181829825044,-104.77268717251718,1936.800048828125,0.550000011920929,103
2024-12-06T15:07:13.000Z,38.888173615559936,-104.77266328409314,1936.800048828125,2.819999933242798,103
2024-12-06T15:07:14.000Z,38.88816607184708,-104.77263604290783,1936.5999755859375,5.320000171661377,102
2024-12-06T15:07:15.000Z,38.888157019391656,-104.77260008454323,1936.4000244140625,8.600000381469727,104
2024-12-06T15:07:16.000Z,38.88815182261169,-104.77257468737662,1936.4000244140625,10.869999885559082,104
2024-12-06T15:07:17.000Z,38.888145200908184,-104.77254685945809,19